In [1]:
from visual_networks import *
from sample_networks import *
import torch


# Generate some random data
x = torch.randn(50, 10)

# Initialize and run the model
model = SimpleLinearModel()
y_pred = model(x)


In [2]:
from visual import *
network = Network(model)  # Example: 3 layers with 3, 4, and 2 neurons respectively
thread = Thread(target=run_glfw_window, args=(network,))
thread.start()

-0.11976263


tensor([[-0.7394, -0.3357,  1.3408,  ...,  1.0387,  0.0947, -1.2005],
        [-2.8897,  0.8601,  0.9920,  ..., -0.6005,  0.3983, -0.7590],
        [-0.0505,  0.2771, -0.9249,  ...,  0.2406,  0.6189,  0.2877],
        ...,
        [-1.0813, -2.7656,  0.5796,  ...,  0.8079, -0.0645,  0.0634],
        [ 0.6240, -0.9443,  0.2103,  ...,  0.3921,  0.9234,  0.4741],
        [ 0.5196,  0.8818, -0.5535,  ..., -1.3143, -3.1685, -1.0717]]) tensor([-2.3928e+00, -3.5796e+00,  1.9439e+00, -1.3783e+00,  4.9137e-01,
        -9.3395e-01,  5.3940e-01,  1.1893e-01,  3.5005e-01,  1.8123e-01,
        -2.9660e-01, -2.6508e-01, -2.4052e+00, -1.5213e+00, -2.9151e+00,
        -1.9164e+00,  5.7917e+00, -1.0360e+00, -2.0223e+00,  1.5485e+00,
        -1.1046e+00,  1.4079e+00,  2.5922e+00, -4.3413e-01, -2.2353e+00,
         1.1466e+00, -2.1167e+00,  2.4540e+00,  2.3906e+00,  2.3245e+00,
         2.0978e+00,  9.8692e-01, -1.6310e+00,  3.0052e+00, -2.3284e+00,
         2.0924e+00,  7.9470e-02, -1.8076e-01, -4.1014e

In [4]:
from tracking_objects import *
tracking_object()

FPS: 63
FPS: 60
0.89125 0.99
0.89125 0.9816666666666667
0.88375 0.805
0.88375 0.795
0.8775 0.6266666666666667
0.87875 0.495
0.88 0.4866666666666667
0.88875 0.39666666666666667
0.89 0.39166666666666666
0.90125 0.33
FPS: 60
0.91125 0.28833333333333333
0.91125 0.2866666666666667
0.92125 0.25666666666666665
0.935 0.23333333333333334
0.95375 0.21333333333333335
0.95625 0.21
0.97625 0.19
0.9775 0.18833333333333332
0.9975 0.16833333333333333
0.995 0.041666666666666664
0.99375 0.041666666666666664
0.98625 0.041666666666666664
0.985 0.041666666666666664
0.97875 0.041666666666666664
0.9775 0.04
0.97375 0.03166666666666667
0.97 0.023333333333333334
0.96625 0.015
0.96375 0.006666666666666667
0.96 0.0016666666666666668


c:\ProgramData\Miniconda3\lib\site-packages\glfw\__init__.py:916: GLFWError: (65537) b'The GLFW library is not initialized'
  warnings.warn(message, GLFWError)
